In [ ]:
import sys

from builtins import chr
from PIL      import Image, ImageDraw, ImageFont

# Fonts and Canvas Logo

In [ ]:
font_text  = ImageFont.truetype('/Users/marcuspompeu/Library/Fonts/Roboto-Bold.ttf', size=32)
font_arrow = ImageFont.truetype('/Users/marcuspompeu/Library/Fonts/Wingdings 3.ttf', size=32)

In [ ]:
image_canvas_logo = Image.open('./Canvas Logo.png', 'r')

# KPIs data

In [ ]:
kpi_active_students = 21101169
kpi_active_teachers = 1424621
kpi_concurrent_users = 1810470
kpi_peak_concurrent_users = '4.13M'
kpi_nps = 94
#
kpi_average_uptime = 0.9992
kpi_support_tickets = 26126
kpi_avg_dau = 5057765
kpi_active_courses = 3299583
kpi_student_submissions = 3104001
kpi_late_submissions = 2504001
kpi_using_k5 = 24
kpi_using_new_outcomes = 1421

def agg_title(title, texts):
    def agg_texts(texts):
        return \
            {
                t[0]:
                    {
                        'length': int(font_text.getlength(t[0])),
                        'height': int(font_text.getbbox(t[0])[3] - font_text.getbbox(t[0])[1] + 1),
                        'raising': t[1],
                        'green': t[2]
                    }
                for t
                in texts
            }

    return \
        {
            'length': int(font_text.getlength(title)),
            'height': int(font_text.getbbox(title)[3] - font_text.getbbox(title)[1] + 1),
            'KPIs': agg_texts(texts)
        }

titles = \
    {
        t[0]: agg_title(t[0], t[1])
        for t
        in
            [
                (
                    'GLOBAL MONTHLY -',
                    [
                        (f'Active Students: {kpi_active_students:,}', True, True),
                        (f'Active Teachers: {kpi_active_teachers:,}', True, True),
                        (f'Avg. Concurrent Users: {kpi_concurrent_users:,}', False, False),
                        (f'Peak Concurrent Users: {kpi_peak_concurrent_users}', True, True),
                        (f'Net Promoter Score: {kpi_nps:,}', True, True)
                    ]
                ),
                (
                    'GLOBAL LAST 7 DAYS -',
                    [
                        (f'Average Uptime: {kpi_average_uptime:.2%}', True, True),
                        (f'Support Tickets Filed: {kpi_support_tickets:,}', False, False),
                        (f'Avg. DAU: {kpi_avg_dau:,}', True, True),
                        (f'Active Courses: {kpi_active_courses:,}', True, True),
                        (f'Student Submissions: {kpi_student_submissions:,}', True, True),
                        (f'Late/Missing Submissions: {kpi_late_submissions:,}', True, False),
                        (f'Using K-5: {kpi_using_k5:,}', True, True),
                        (f'Using New Outcomes: {kpi_using_new_outcomes:,}', True, True)
                    ]                    
                )
            ]
    }

# Widths and heights

In [ ]:
width_logo  = image_canvas_logo.width
margin_logo = 16
#
width_arrow = int(font_arrow.getlength(chr(112)))
#
spacer_group  = 32
spacer_ticker = 16
width_group_spacer  = 32
width_ticker_spacer = 16

width_total = \
    int(
        2 +
        #
        margin_logo + width_logo + margin_logo +
        #
        sum(
            spacer_group + d['length'] + spacer_group
            for d
            in titles.values()
        ) +
        #
        sum(
            [
                k['length'] + spacer_ticker + width_arrow + spacer_group
                for t
                in titles.values()
                for k
                in t['KPIs'].values()
            ]
        )
    )

In [ ]:
height_logo      = image_canvas_logo.height
height_arrow     = font_arrow.getbbox(chr(112))[3] - font_arrow.getbbox(chr(112))[1] + 1
height_max_texts = max(k['height'] for t in titles.values() for k in t['KPIs'].values())

# Gradient helper (https://stackoverflow.com/a/32532502/206413)

In [ ]:
class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y

class Rect(object):
    def __init__(self, x1, y1, x2, y2):
        minx, maxx = (x1,x2) if x1 < x2 else (x2,x1)
        miny, maxy = (y1,y2) if y1 < y2 else (y2,y1)
        self.min = Point(minx, miny)
        self.max = Point(maxx, maxy)

    width  = property(lambda self: self.max.x - self.min.x)
    height = property(lambda self: self.max.y - self.min.y)


def gradient_color(minval, maxval, val, color_palette):
    """ Computes intermediate RGB color of a value in the range of minval
        to maxval (inclusive) based on a color_palette representing the range.
    """
    max_index = len(color_palette)-1
    delta = maxval - minval
    if delta == 0:
        delta = 1
    v = float(val-minval) / delta * max_index
    i1, i2 = int(v), min(int(v)+1, max_index)
    (r1, g1, b1), (r2, g2, b2) = color_palette[i1], color_palette[i2]
    f = v - i1
    return int(r1 + f*(r2-r1)), int(g1 + f*(g2-g1)), int(b1 + f*(b2-b1))

def horz_gradient(draw, rect, color_func, color_palette):
    minval, maxval = 1, len(color_palette)
    delta = maxval - minval
    width = float(rect.width)  # Cache.
    for x in range(rect.min.x, rect.max.x+1):
        f = (x - rect.min.x) / width
        val = minval + f * delta
        color = color_func(minval, maxval, val, color_palette)
        draw.line([(x, rect.min.y), (x, rect.max.y)], fill=color)

def vert_gradient(draw, rect, color_func, color_palette):
    minval, maxval = 1, len(color_palette)
    delta = maxval - minval
    height = float(rect.height)  # Cache.
    for y in range(rect.min.y, rect.max.y+1):
        f = (y - rect.min.y) / height
        val = minval + f * delta
        color = color_func(minval, maxval, val, color_palette)
        draw.line([(rect.min.x, y), (rect.max.x, y)], fill=color)

# Generate the Ticker

In [ ]:
ticker_height = 96

image      = Image.new('RGBA', (width_total, 96), color='black')
image_draw = ImageDraw.Draw(image)

def do_draw():
    def do_canvas_logo():
        nonlocal col
        
        image.paste(image_canvas_logo, (2 + margin_logo, (ticker_height - height_logo) // 2))
        
        col += margin_logo + image_canvas_logo.width + margin_logo

    def do_draw_text(text, height, font, fill=None):
        image_draw.text((col, (ticker_height - height) // 2), text, font=font, fill=fill)
        
    def do_title(item):
        nonlocal col
        
        text   = item[0]
        length = item[1]['length']
        height = item[1]['height']
        
        horz_gradient(
            image_draw,
            Rect(
                col, 0,
                col + length + spacer_group, image.height
            ),
            gradient_color,
            [
                (0x3d, 0xBa, 0xe2),
                (0, 0, 0)
            ]
        )
        
        col += spacer_group
        
        do_draw_text(text, height, font_text)
        
        col += length + spacer_group        
        
    def do_text(item):
        nonlocal col
        
        text   = item[0]
        length = item[1]['length']
        height = item[1]['height']
        
        do_draw_text(text, height, font_text)
        
        col += length + spacer_ticker

    def do_ticker(item):
        nonlocal col
        
        raising = item[1]['raising']
        
        text  = chr(112 + (0 if raising else 1))
        color = (0, 172, 24) if raising else (238, 6, 18)

        do_draw_text(text, height_arrow, font_arrow, color)
        
        col += width_arrow
        
    def do_group_spacer():
        nonlocal col
        
        col += spacer_group // 2

        image_draw.rectangle((col, (image.height - height_max_texts) // 2, col + 1, (image.height - height_max_texts) // 2 + height_max_texts), fill=(255, 255, 255))

        col += spacer_group // 2

    col = 2
    
    do_canvas_logo()
    
    for title in titles.items():
        do_title(title)
        kpis = title[1]['KPIs']
        
        for i, item in enumerate(kpis.items()):
            do_text(item)
            do_ticker(item)
            
            if i != len(kpis) - 1:
                do_group_spacer()
                
        col += spacer_group

do_draw()    

In [ ]:
image_draw.rectangle((0,               0,                1,               image.height),     fill=(255, 255, 255))
image_draw.rectangle((0,               0,                image.width,     1),                fill=(255, 255, 255))
image_draw.rectangle((0,               image.height - 2, image.width,     image.height - 1), fill=(255, 255, 255))
image_draw.rectangle((image.width - 2, 0,                image.width - 1, image.height),     fill=(255, 255, 255))

# Output

In [ ]:
image.save('ticker.png')